# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770919025522                   -0.52    9.0         
  2   -2.772150136963       -2.91       -1.32    1.0    206ms
  3   -2.772170350192       -4.69       -2.46    1.0    215ms
  4   -2.772170662336       -6.51       -3.19    1.0    181ms
  5   -2.772170722878       -7.22       -4.35    2.0    284ms
  6   -2.772170722937      -10.23       -4.48    1.0    185ms
  7   -2.772170723012      -10.13       -5.35    1.0    194ms
  8   -2.772170723014      -11.63       -5.68    1.0    233ms
  9   -2.772170723015      -12.13       -6.71    2.0    229ms
 10   -2.772170723015   +  -15.05       -6.89    1.0    239ms
 11   -2.772170723015   +  -14.01       -7.33    1.0    245ms
 12   -2.772170723015      -14.07       -8.39    2.0    252ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735578504242164

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770752871426                   -0.52    9.0         
  2   -2.772059071763       -2.88       -1.33    1.0    172ms
  3   -2.772082839514       -4.62       -2.41    1.0    214ms
  4   -2.772083324722       -6.31       -3.09    1.0    180ms
  5   -2.772083417715       -7.03       -4.76    2.0    261ms
  6   -2.772083417798      -10.08       -4.79    2.0    218ms
  7   -2.772083417807      -11.03       -5.46    1.0    210ms
  8   -2.772083417811      -11.47       -6.30    2.0    232ms
  9   -2.772083417811      -13.13       -7.04    1.0    231ms
 10   -2.772083417811      -14.03       -8.07    2.0    247ms

Polarizability via ForwardDiff:       1.7725349422078385
Polarizability via finite difference: 1.7735578504242164
